In [ ]:
# compute distances
dim = len(features)
distances = np.zeros([dim,dim])
for u in tqdm(np.arange(dim)):
    for v in list(range(u,dim)):
                if (u==v):
                    distances[u][v] =0
                else:
                    dist = spatial.distance.cosine(features.iloc[u],features.iloc[v])
                    distances[u][v]=dist
                    distances[v][u]=dist

                    
# compute weights
kernel_width = distances.mean()
weights = np.zeros([dim,dim])
for u in np.arange(dim):
    for v in list(range(u,dim)):
        if u==v : 
            weights[u][v]=0
        else:
            weight = compute_weight_from_dist(distances[u][v],kernel_width)
            weights[u][v] = weight
            weights[v][u] = weight
            
# Select best n edges
NEIGHBORS = 100


weights_=weights.copy()
weights = np.zeros([dim,dim])
for u in np.arange(dim):
    to_keep = np.argsort(weights_[u])[-NEIGHBORS:]
    for v in to_keep:
        w = weights_[u][v]
        weights[u][v] = w
        weights[v][u] = w
plot(weights, axes[:, 1])

#degrees
degrees = [np.sum(weights[vertex]) for vertex in np.arange(dim)]

# Normalised laplacian
# D to the power -0.5
sqrt_degrees = list(map(lambda x: 1.0/math.sqrt(x),degrees))
D_sqrt = np.diag(sqrt_degrees)

# constuct D
D = np.diag(degrees)

# construct laplacian
laplacian = D - weights

# normalised laplacian
laplacian = D_sqrt.dot(laplacian).dot(D_sqrt)

# eighen vectors
eigenvalues, eigenvectors = sparse.linalg.eigsh(laplacian,k=10,which='SA')

# Plot
x = eigenvectors[:,1]
y = eigenvectors[:,2]
plt.scatter(x, y, c=genres, cmap='RdBu', alpha=0.5);

# Change a sign into a label in {0,1}
def sign_to_genre (i):
    if i== -1 : return 0
    else: return 1
labels = list(map(sign_to_genre,np.sign(eigenvectors[:,1])))

# get error
err = len([i for i in np.arange(dim) if labels[i] != genres[i]])
print('{} errors ({:.2%})'.format(err, err/len(labels)))